In [36]:
import h5py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import numpy as np


In [ ]:
#load the training data set
with h5py.File(rf'C:\Users\Wahaj Sajid\Desktop\Datasets\Finger Signs Data sets\Signs_Data_Training.h5', 'r') as f:
    print(f.keys())
    x_train = f['train_set_x'][:]
    y_train = f['train_set_y'][:]

    print(x_train.shape)
    print(y_train.shape)

<KeysViewHDF5 ['list_classes', 'train_set_x', 'train_set_y']>
(1080, 64, 64, 3)
(1080,)


In [12]:
#load the testing dataset
with h5py.File(rf'C:\Users\Wahaj Sajid\Desktop\Datasets\Finger Signs Data sets\Signs_Data_Testing.h5', 'r' ) as f:
    print(f.keys())

    x_test = f['test_set_x'][:]
    y_test = f['test_set_y'][:]

print(x_test.shape)
print(y_test.shape)

<KeysViewHDF5 ['list_classes', 'test_set_x', 'test_set_y']>
(120, 64, 64, 3)
(120,)


# Below is the implementation of model trained on Gray Scale images (128, 128, 1)

In [71]:
#changing the shape of train images and resizing them
train_gray_images = []
for img in x_train:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (128, 128))
    img = img/255.0
    train_gray_images.append(img.reshape(128, 128, 1))

train_gray_images = np.array(train_gray_images)

In [76]:
#changing the shape of test images and resizing them
test_gray_images = []
for img in x_test:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (128, 128))
    img = img/255.0
    test_gray_images.append(img.reshape(128, 128, 1))
test_gray_images = np.array(test_gray_images)

In [207]:
#train the images on the gray scale images
model_1 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape = (128, 128, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),

    Dense(units=25, activation='relu'),
    Dense(units=6, activation='softmax')
])

model_1.compile(optimizer = 'adam', loss = SparseCategoricalCrossentropy(), metrics = ['accuracy'])
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_1.fit(train_gray_images, y_train, epochs=55, validation_split=0.2, batch_size=32)

Epoch 1/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 289ms/step - accuracy: 0.1807 - loss: 1.8615 - val_accuracy: 0.2593 - val_loss: 1.7887
Epoch 2/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 274ms/step - accuracy: 0.2723 - loss: 1.7725 - val_accuracy: 0.3519 - val_loss: 1.6758
Epoch 3/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 274ms/step - accuracy: 0.4085 - loss: 1.5451 - val_accuracy: 0.5046 - val_loss: 1.3612
Epoch 4/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 278ms/step - accuracy: 0.6167 - loss: 1.1069 - val_accuracy: 0.5880 - val_loss: 1.1649
Epoch 5/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - accuracy: 0.7228 - loss: 0.8420 - val_accuracy: 0.6343 - val_loss: 1.0213
Epoch 6/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 277ms/step - accuracy: 0.7559 - loss: 0.7173 - val_accuracy: 0.6667 - val_loss: 0.9935
Epoch 7/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 278ms/step - accuracy: 0.8315 - loss: 0.5233 - val_accuracy: 0.6667 - val_loss: 0.9651
Epoch 8/55
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 283ms/step - accuracy: 0.8886 - loss: 0.3591 - val_accuracy: 0

In [179]:
#evaluate the gray_scale_images_trained_model on a training data 
loss, accuracy = model_1.evaluate(train_gray_images, y_train)
print(f"Loss", loss)
print(f"Accuracy", accuracy)

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9607 - loss: 0.1810
Loss 0.28585392236709595
Accuracy 0.9203703999519348


In [180]:
#evaluate the rgb_images_trained_model on a testing data
loss, accuracy = model_1.evaluate(test_gray_images, y_test)
print(f"Loss", loss)
print(f"Accuracy", accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8290 - loss: 0.5474
Loss 0.5660637617111206
Accuracy 0.824999988079071


In [204]:
#test the gray_scale_images_trained_model
path = rf"C:\Users\Wahaj Sajid\Desktop\5.jpeg"
img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img,(128,128))
img = img/255.0
img = img.reshape(128, 128, 1)

img = np.expand_dims(img, axis=0)

prediction = model_1.predict(img)

predict = np.argmax(prediction)
print(predict)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
4
[[2.6767418e-07 5.2181264e-08 2.7868373e-07 1.5276831e-02 9.8209298e-01
  2.6296673e-03]]


# Below is the implementation of model trained on RGB images (64, 64, 3)

In [189]:
#train the model on the rgb
model = Sequential([
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),

    Dense(units=25, activation='relu'),
    Dense(units=6, activation='softmax')
])

model.compile(optimizer = 'adam', loss = SparseCategoricalCrossentropy(), metrics = ['accuracy'])
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(x_train, y_train, epochs=50,validation_split=0.2, batch_size=35)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.1753 - loss: 79.9527 - val_accuracy: 0.2037 - val_loss: 1.8062
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.3450 - loss: 1.5833 - val_accuracy: 0.4861 - val_loss: 1.3101
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.6466 - loss: 1.0314 - val_accuracy: 0.5370 - val_loss: 1.1119
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7872 - loss: 0.6270 - val_accuracy: 0.6944 - val_loss: 0.9979
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8931 - loss: 0.3221 - val_accuracy: 0.6250 - val_loss: 1.0605
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9392 - loss: 0.1748 - val_accuracy: 0.7500 - val_loss: 0.9278
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.9833 - loss: 0.0747 - val_accuracy: 0.7639 - val_loss: 1.0099
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.9927 - loss: 0.0428 - val_accuracy: 0.7407 - 

In [190]:
#evaluate the rgb_images_trained_model on a training data
loss, accuracy = model.evaluate(x_train, y_train)
print(f"Loss", loss)
print(f"Accuracy", accuracy)

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9900 - loss: 0.0574
Loss 0.32830363512039185
Accuracy 0.9425926208496094


In [191]:
#evaluate the rgb_images_trained_model on a testing data
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss", loss)
print(f"Accuracy", accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7738 - loss: 1.4126
Loss 1.3463404178619385
Accuracy 0.7833333611488342


In [202]:
#test the rgb_images_trained_model
path = rf"C:\Users\Wahaj Sajid\Desktop\4.jpeg"
img = cv2.imread(path,cv2.IMREAD_COLOR_RGB)
img = cv2.resize(img,(64,64))
img = img/255.0
img = img.reshape(64, 64, 3)

img = np.expand_dims(img, axis=0)

prediction = model.predict(img)

predict = np.argmax(prediction)
print(predict)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
2
[[0.15522233 0.17337298 0.18157774 0.16389963 0.16957322 0.15635405]]
